In [ ]:
import numpy as np
import pandas as pd 
import yaml

import data_api
import helpers
import BrownsvilleAPI

## 1. Loading the datasets

In [ ]:
# Load the configuration files with all the credentials for the Socrata API
with open("./config.yaml", 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    app_token, username, password = config["sodapy"].values()

In [ ]:
# Create the data_api client and set the number of records to be returned

client = data_api.Client(
    app_token=app_token,
    username=username,
    password=password
)
num_records = 100

# Note: since we are not providing an api token to the client, it is best to keep the number 
# of records returned low

## 311 Service Requests from 2010 to Present

In [ ]:
df_311 = client.load_311(
    fetch_all=True, 
    select="created_date, closed_date, complaint_type, descriptor, x_coordinate_state_plane,    \
                status, due_date, bbl, y_coordinate_state_plane, latitude, longitude", 
    where=
        "agency IN ('DOB', 'HPD') AND incident_zip IN ('11212', '11233')"
)
df_311

In [ ]:
df_311.shape

In [ ]:
df_complaint_problems = client.load_complaint_problems(
    fetch_all=True, 
    where="unittypeid=92"
)
df_complaint_problems.head()

In [ ]:
df_complaint_problems.shape

In [ ]:
# NOTE: the DOB Complaints Received dataset zip_code column has extra spaces appended to the string, so we are using the starts_with function from the SoQL API to filter the results.

# starts_with documentation: https://dev.socrata.com/docs/functions/starts_with.html

df_dob_complaints = client.load_dob_complaints(
    fetch_all=True,
    select=
        "complaint_number, date_entered, house_number, house_street,            \
            bin, complaint_category, unit, disposition_date, disposition_code,  \
            inspection_date, dobrundate",
    where=
        "starts_with(zip_code, '11212') OR                                      \
            starts_with(zip_code, '11233')"
) 
df_dob_complaints.head()

In [ ]:
df_dob_complaints.shape

In [ ]:
print(min(df_dob_complaints["date_entered"]), max(df_dob_complaints["date_entered"]))

In [ ]:
df_311 = client.load_311(fetch_all=True, select=["Created Date"])
# df_complaint_problems = client.load_complaint_problems(limit=num_records)
# df_dob_omplaints = client.load_dob_complaints(limit=num_records)

In [ ]:
df_311.head()

In [ ]:
df_complaint_problems.head()

In [ ]:
df_dob_complaints.head()

In [ ]:
df_brownsville = pd.read_csv('./data/brownsville.csv')
df_brownsville.head()

In [ ]:

dates = pd.DatetimeIndex(df_brownsville["StatusDate"]).month.astype("Int64")
date_counts = dates.value_counts()

print(f"Winter: {date_counts.loc[[12, 1, 2]].sum()}")
print(f"Spring: {date_counts.loc[[3,  4, 5]].sum()}")
print(f"Summer: {date_counts.loc[[6,  7, 8]].sum()}")
print(f"Autumn: {date_counts.loc[[9, 10, 11]].sum()}")


In [ ]:
n = input('type something here: ')
print(n)

In [ ]:
date_counts.sort_index()

In [ ]:
dates =  pd.to_datetime(df_brownsville["StatusDate"])
dates.head()
df_dates = (df.assign(date=dates)
        .groupby(['user_id', pd.Grouper(key='date', freq='3D')])
        .sum()
        .reset_index())
# print(df_dates)

## 2. Extracting useful columns

In [ ]:
records= client._client.get("uwyv-629c", where="zip='11212' OR zip='11233'", limit=10000, offset=2000)
df_hc = pd.DataFrame.from_records(records)
# df_hc["complaintid"] = df_hc["complaintid"].astype("int64")
df_hc.head()

In [ ]:
records= client._client.get("a2nx-4u46", limit=10000, offset=28000)
df_cp = pd.DataFrame.from_records(records)
# df_cp["complaintid"] = df_cp["complaintid"].astype("int64")
df_cp.head()

In [ ]:
brownsville = pd.merge(df_hc, df_cp, on=["complaintid", "statusid", "status", "statusdate"])
brownsville = brownsville[["zip", "borough", "boroughid", "block", "lot",  "buildingid", "housenumber", "streetname", "apartment", "problemid", "complaintid", "unittypeid", "unittype", "spacetypeid", "spacetype", "typeid", "type", "majorcategoryid", "majorcategory", "minorcategoryid", "minorcategory", "codeid", "code", "statusid", "status", "receiveddate", "statusdate", "statusdescription" ]]
brownsville.head()

In [ ]:
print(brownsville.shape, brownsville.columns, sep='\n')


In [ ]:
print(brownsville.shape, brownsville.columns, sep='\n')

In [ ]:
brownsville[['statusid_y', 'status_y', 'statusdate_y','statusid_x', 'status_x', 'statusdate_x']]
x = brownsville[['statusid_x', 'status_x', 'statusdate_x']]
y = brownsville[['statusid_y', 'status_y', 'statusdate_y']]


In [ ]:
housing_problems = pd.read_csv("./data/housing-maintenance-code-complaints-raw.csv")
complaint_problems = pd.read_csv("./data/complaint-problems-raw.csv")


In [ ]:
housing_problems.head()

In [ ]:
complaint_problems.head()


In [ ]:
complaint_problems.head()

In [ ]:
print(housing_problems.shape, complaint_problems.shape)

In [ ]:
df = pd.merge(housing_problems, complaint_problems, on=["complaintid"])
df.head()

In [ ]:
df.columns

In [ ]:
df.to_csv("./data/brownsville-raw.csv")

In [ ]:
bv.head()

In [ ]:
client.close()

In [ ]:
import BrownsvilleAPI
b = BrownsvilleAPI.Brownsville()

In [ ]:
b.data.head()

In [ ]:
b.get_feature_occurrences_by_building()

In [ ]:
building_common_complaints = []
for _id in set(b.data["buildingid"]):
    common_complaints = b.get_feature_occurrences_by_building(
                                                            _id,
                                                            by=["majorcategory", "minorcategory"],
                                                            find_all=True)

    building_common_complaints.append((_id, common_complaints))
building_common_complaints.sort(key=lambda e: e[1].values.sum(), reverse=True)

In [ ]:
for building in building_common_complaints[:10]:
    id_, complaints = building
    major_category, minor_category = complaints.index[0]
    num_complaints = complaints.values.sum()

    print("Building ID:", id_)
    print("Most common major category:", major_category)
    print("Most common minor category:", minor_category)
    print("Number of complaints:", num_complaints)

    print()

In [ ]:
b.data.shape

In [ ]:
b.data.head()

In [ ]:
b.data.dtypes

In [ ]:
b.data["unittypeid"].value_counts()